In [25]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 29, Finished, Available, Finished)

In [29]:
%%sql
create table if not EXISTS Project_Lakehouse.gold_product
(
   Product_ID long,
   Product_Category string,
   Product string,
   Created_TS timestamp,
   Modified_TS timestamp 
)
USING DELTA                             

StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 33, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [30]:
Max_Date = spark.sql("""
SELECT coalesce(max(Modified_TS),'1990-01-01') FROM Project_Lakehouse.gold_product
""").first()[0]

StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 34, Finished, Available, Finished)

In [31]:
Max_Date

StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 35, Finished, Available, Finished)

'1990-01-01'

In [32]:
df_bronze = spark.sql("""
SELECT distinct Product_Category, Product
FROM Project_Lakehouse.bronze_sales WHERE Modified_TS>'{}'
""".format(Max_Date))

StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 36, Finished, Available, Finished)

In [33]:
Max_ID=spark.sql("select coalesce(max(Product_ID),'0') from Project_Lakehouse.gold_product").first()[0]

StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 37, Finished, Available, Finished)

In [34]:
df_final = df_bronze.withColumn("Product_ID",monotonically_increasing_id()+Max_ID+1)

StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 38, Finished, Available, Finished)

In [35]:
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 39, Finished, Available, Finished)

In [36]:
%%sql
MERGE INTO Project_Lakehouse.gold_product AS gp
USING ViewProduct AS vp
ON gp.Product = vp.Product 
   AND gp.Product_Category = vp.Product_Category
WHEN MATCHED THEN
    UPDATE SET
        gp.Modified_TS = CURRENT_TIMESTAMP()
WHEN NOT MATCHED THEN
    INSERT (
        Product_ID,
        Product_Category,
        Product,
        Created_TS,
        Modified_TS
    )
    VALUES (
        vp.Product_ID,
        vp.Product_Category,
        vp.Product,
        CURRENT_TIMESTAMP(),
        CURRENT_TIMESTAMP()
    )


StatementMeta(, 079b3531-492b-4b57-8ede-1154b0ce7a4c, 40, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>